In [45]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_full = len(dm.train_dataloader())
l_val_full = len(dm.val_dataloader())
l_test_full = len(dm.test_dataloader())
print(l_train_full, l_val_full, l_test_full)


Seed set to 0


took : 15.291179895401001
loading 109.07367300987244
5038 560 1520


In [46]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/fast_config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_fast = len(dm.train_dataloader())
l_val_fast = len(dm.val_dataloader())
l_test_fast = len(dm.test_dataloader())
print(l_train_fast, l_val_fast, l_test_fast)

Seed set to 0


took : 2.798424005508423
loading 14.54524302482605
916 102 236


In [41]:
print(l_train_fast/l_train_full)
print(l_val_fast/l_val_full)
print(l_test_fast/l_test_full)

0.18181818181818182
0.18214285714285713
0.15526315789473685


In [ ]:
dummpy = torch.randn(8, 52, 2304)
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/experiments/pretrained_ae_dlinear_indc_indp/config.yaml").dlinear
model = DLinear(cfg)
model2 = DLinear2(cfg)
model.train()
model2.train()
state_dict = model.state_dict()
model2.load_state_dict(state_dict)
out1 = model(dummpy)
out2 = model2(dummpy)
#check all parameters are same
flag = True
for p1, p2 in zip(model.parameters(), model2.parameters()):
    if not torch.allclose(p1, p2, atol=1e-4):
        flag = False
        break
print("Parameters match:", flag)
print(out1.shape, out2.shape)
print(torch.allclose(out1, out2, atol=1e-12))  #
target = torch.randn(8, 48, 2304) 
loss1 = F.mse_loss(out1, target)
loss2 = F.mse_loss(out2, target)

loss1.backward()
loss2.backward()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)
optim2 = torch.optim.SGD(model2.parameters(), lr=1e-3)
optim.step()
optim2.step()

# Check if the gradients are the same
flag = True
for (name1, p1), (name2, p2) in zip(model.named_parameters(), model2.named_parameters()):
    if p1.grad is None or p2.grad is None:
        print(f"Gradient is None for: {name1}, {name2}")
        assert p1.grad is None and p2.grad is None, "both gradients should be None"
        print("skipping gradient check for None gradients")
        continue
    if not torch.allclose(p1.grad, p2.grad, atol=1e-4):
        flag = False
        break
print("Gradients match:", flag)

out1_again = model(dummpy)
out2_again = model2(dummpy)
print(torch.allclose(out1_again, out2_again, atol=1e-12))  # should be True after training

Parameters match: True
torch.Size([8, 48, 2304]) torch.Size([8, 48, 2304])
True
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient

In [9]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.act1 = nn.GELU()
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        # Skip connection path
        # If stride is not 1 or input/output channels are different, we need to project the skip connection
        # to match the output dimensions of the main path.
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
            
        self.act2 = nn.GELU()

    def forward(self, x):
        shortcut = self.shortcut(x)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += shortcut
        return self.act2(out)


class UpsampleBlock(nn.Module):

    def __init__(self, in_ch, out_ch, scale_factor):
        super().__init__()
        # Using 'nearest' mode for upsampling avoids checkerboard artifacts that can be
        # caused by transposed convolutions.
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='nearest')
        # After upsampling, we use a residual block to learn and refine features.
        self.resblock = ResidualBlock(in_ch, out_ch, stride=1)

    def forward(self, x):
        return self.resblock(self.upsample(x))

class ConvAutoencoder(nn.Module):
    def __init__(self, in_ch=1, latent_dim=512):
        super().__init__()
        
        # Encoder: 384 → 96 → 48 → 24 → 6 → 1
        self.enc1 = ResidualBlock(in_ch,  64, stride=4)  # 384→96
        self.enc2 = ResidualBlock(64,    128, stride=2)  # 96→48
        self.enc3 = ResidualBlock(128,   256, stride=2)  # 48→24
        self.enc4 = ResidualBlock(256,   512, stride=4)  # 24→6
        self.enc5 = ResidualBlock(512,  1024, stride=6)  # 6→1

        self.flatten = nn.Flatten()             # (B,1024,1,1) → (B,1024)
        self.fc_enc  = nn.Linear(1024, latent_dim)

        self.fc_dec  = nn.Linear(latent_dim, 1024)
        self.unflatten = nn.Unflatten(1, (1024, 1, 1)) # (B,1024) → (B,1024,1,1)
        
        self.dec_init_conv = ResidualBlock(1024, 1024, stride=1)

        # Decoder: 1 → 6 → 24 → 48 → 96 → 384
        self.dec1 = UpsampleBlock(1024, 512, scale_factor=6)  # 1→6
        self.dec2 = UpsampleBlock(512,  256, scale_factor=4)  # 6→24
        self.dec3 = UpsampleBlock(256,  128, scale_factor=2)  # 24→48
        self.dec4 = UpsampleBlock(128,   64, scale_factor=2)  # 48→96
        
        self.final_upsample = nn.Upsample(scale_factor=4, mode='nearest') # 96→384
        self.final_conv = nn.Conv2d(64, in_ch, kernel_size=3, stride=1, padding=1)


    def encode(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)
        x = self.flatten(x)
        z = self.fc_enc(x)
        return z

    def decode(self, z):
        x = self.fc_dec(z)
        x = self.unflatten(x)
        x = self.dec_init_conv(x)
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        x = self.dec4(x)
        x = self.final_upsample(x)
        x = self.final_conv(x)
        return x

    def forward(self, x):
        z = self.encode(x)
        reconstruction = self.decode(z)
        return reconstruction

if __name__ == "__main__":
    model = ConvAutoencoder(in_ch=1, latent_dim=512)
    
    input_tensor = torch.randn(4, 1, 384, 384)
    
    output_tensor = model(input_tensor)
    
    # Check the output shape
    print(f"Input Shape:  {input_tensor.shape}")
    print(f"Output Shape: {output_tensor.shape}")
    assert input_tensor.shape == output_tensor.shape

    latent_vector = model.encode(input_tensor)
    print(f"Latent Vector Shape: {latent_vector.shape}")
    assert latent_vector.shape == (4, 512)
    
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters: {num_params:,}")

Input Shape:  torch.Size([4, 1, 384, 384])
Output Shape: torch.Size([4, 1, 384, 384])
Latent Vector Shape: torch.Size([4, 512])
Number of trainable parameters: 49,577,921
